# Algoritmo de Goertzel: Análisis en el Dominio de Frecuencia

El **algoritmo de Goertzel** es un método eficiente para el análisis de frecuencias individuales en una señal discreta. Es especialmente útil en aplicaciones donde se requiere detectar solo una o unas pocas frecuencias específicas dentro de una señal, como en el caso del reconocimiento de tonos DTMF en telefonía.

## Funcionamiento del Algoritmo

Goertzel es una transformación discreta de Fourier (DFT) modificada para calcular el contenido de frecuencia en un punto específico, evitando así el cálculo de todo el espectro de frecuencias como se hace en la FFT (Fast Fourier Transform). Esto lo hace particularmente útil cuando solo queremos conocer la amplitud o fase de algunas frecuencias definidas.

### Fórmulas Básicas

1. **Frecuencia objetivo**: En el algoritmo de Goertzel, seleccionamos una frecuencia objetivo \( f \) (en Hz) y determinamos su correspondiente índice \( k \) en el dominio de frecuencia utilizando la siguiente relación:

   $k = \frac{N \cdot f}{f_s}$

   donde \( N \) es el número de muestras de la señal y \( f_s \) es la frecuencia de muestreo.

3. **Recurrencia de Goertzel**: Para cada índice \( n \) en la señal, se calcula el siguiente valor de la secuencia utilizando una recurrencia de segundo orden:

   $s[n] = x[n] + 2 \cdot \cos\left(\frac{2 \pi k}{N}\right) \cdot s[n - 1] - s[n - 2]$
   
   Aquí, $ x[n]$ es la señal de entrada en el tiempo discreto, y $ s[n] $ es la señal transformada.

4. **Cálculo de la Magnitud**: Después de procesar toda la señal, la magnitud en la frecuencia objetivo se obtiene como:

$
   \text{magnitud} = \sqrt{s[N-1]^2 + s[N-2]^2 - 2 \cdot \cos\left(\frac{2 \pi k}{N}\right) \cdot s[N-1] \cdot s[N-2]}
$
## Comparación con FFT

- **Complejidad**: La FFT tiene una complejidad de \( O(N \log N) \) y calcula todo el espectro de frecuencias, mientras que Goertzel tiene una complejidad de \( O(N) \) y se enfoca en una frecuencia específica. Por lo tanto, el algoritmo de Goertzel es ideal para situaciones en las que solo se necesita conocer el contenido de unas pocas frecuencias.
  
- **Aplicaciones**: La FFT es más adecuada para análisis de espectros completos, mientras que Goertzel es óptimo para tareas como el reconocimiento de tonos, detección de frecuencias precisas en señales de comunicación, o afinación de instrumentos en una frecuencia concreta.

## Ejemplo de Aplicación

Si deseamos detectar si una señal contiene una frecuencia específica, como el tono de 330 Hz de la primera cuerda de una guitarra afinada, podemos usar Goertzel para centrar el análisis solo en esa frecuencia.

In [3]:
import numpy as np
import wave
from scipy.io import wavfile

# Implementación de Goertzel para una frecuencia específica
def goertzel(samples, target_freq, sample_rate):
    N = len(samples)
    k = int(0.5 + (N * target_freq) / sample_rate)  # Índice de la frecuencia objetivo
    omega = (2.0 * np.pi * k) / N
    coeff = 2.0 * np.cos(omega)
    
    s_prev = 0.0
    s_prev2 = 0.0
    
    # Algoritmo de recurrencia de Goertzel
    for sample in samples:
        s = sample + coeff * s_prev - s_prev2
        s_prev2 = s_prev
        s_prev = s
    
    # Calcular magnitud de la frecuencia objetivo
    power = s_prev2**2 + s_prev**2 - coeff * s_prev * s_prev2
    return power

# Cargar archivo de audio
def load_wav(filename):
    sample_rate, data = wavfile.read(filename)
    if len(data.shape) > 1:  # Si es estéreo, convertir a mono
        data = np.mean(data, axis=1)
    return sample_rate, data

# Detectar la frecuencia dominante en un rango cercano a la frecuencia objetivo
def detect_frequency(data, sample_rate, target_freq, freq_range=5):
    max_power = 0
    detected_freq = target_freq
    
    # Probar frecuencias en el rango [target_freq - freq_range, target_freq + freq_range]
    for f in np.arange(target_freq - freq_range, target_freq + freq_range, 0.1):
        power = goertzel(data, f, sample_rate)
        if power > max_power:
            max_power = power
            detected_freq = f
    
    return detected_freq

# Configuración
filename = 'D:/UTN/PES/repos/TPS finales/Afinador/Audios/ee2.wav'  # Nombre del archivo WAV
target_freq = 330.0               # Frecuencia de afinación para la primera cuerda
freq_range = 5.0                  # Rango en Hz alrededor de la frecuencia objetivo

# Cargar el archivo de audio
sample_rate, data = load_wav(filename)

# Detectar frecuencia dominante cercana a la frecuencia objetivo
detected_freq = detect_frequency(data, sample_rate, target_freq, freq_range)

# Comparar con la frecuencia objetivo
print(f"Frecuencia detectada: {detected_freq:.2f} Hz")
if abs(detected_freq - target_freq) <= 0.5:
    print("La cuerda está afinada.")
else:
    print(f"La cuerda está desfasada por {detected_freq - target_freq:.2f} Hz.")

# Puede que los armónicos del tono compliquen la solución
# Pensar en usar un filtro pasa banda para obtener solo la fundamental.

Frecuencia detectada: 329.10 Hz
La cuerda está desfasada por -0.90 Hz.
